In [32]:
import numpy as np
from scipy.linalg import svd
from scipy.linalg import qr

# **TT-SVD**

In [97]:
def ttsvd(tensor, eps = 10**(-8), ranks = None):
  T = tensor.copy()
  d = T.ndim
  n = T.shape
  G = []
  rel_eps = (eps * np.linalg.norm(T))**2

  if ranks == None or len(ranks) < d - 1:
    ranks = np.zeros(d - 1, dtype= int)
  ranks = np.insert(ranks, 0, 1)
  for k in range(d - 1):
    unfolding = T.reshape((ranks[k] * n[k], -1))

    U, sigmas, VT = svd(unfolding, full_matrices = False)

    if ranks[k + 1] == 0:
      ranks[k + 1] = sigmas.shape[0]
      s = sigmas[-1]**2;
      while s < rel_eps:
        ranks[k + 1] -= 1
        s += sigmas[-(sigmas.shape[0] - ranks[k + 1])]**2
      ranks[k + 1] += 1
    else:
      ranks[k + 1] = min(sigmas.shape[0], ranks[k+1])

    if k > 0:
      G.append(U[:, :ranks[k + 1]].reshape((ranks[k], n[k], ranks[k + 1])))
    else:
      G.append(U[:, :ranks[k + 1]].reshape((n[k], ranks[k + 1])))
    T = np.diag(sigmas[:ranks[k + 1]]) @ VT[:ranks[k + 1], :]

  G.append(T.reshape((-1, n[-1])))
  ranks = ranks[1:]
  return G, ranks

# **TT-orthogonalization**

In [59]:
def ttort(G):
  for k in range(len(G) - 1):
    unfolding = G[k]
    if k > 0:
      r_left, n_k, r_right = unfolding.shape
      unfolding = unfolding.reshape((-1, r_right))
    Q, R = qr(unfolding, mode= 'economic')
    if k == 0:
      G[k] = Q
    else:
      G[k] = Q.reshape((r_left, n_k, -1))
    G[k + 1] = np.tensordot(R, G[k + 1], axes=(1, 0))

  return G

# **TT-rounding**

In [180]:
def ttround(G, eps = 10**(-8)):
  G = ttort(G)
  d = len(G)
  ranks = np.zeros(d - 1, dtype= int)
  ranks = np.insert(ranks, 0, 1)
  rel_eps = eps * np.linalg.norm(G[-1])**2

  for k in reversed(range(1, d)):
    if k == d - 1:
      unfolding = G[k]
    else:
      r_left, n_k, r_right = G[k].shape
      unfolding = G[k].reshape((r_left, -1))

    U, sigmas, VT = svd(unfolding, full_matrices = False)

    if ranks[k] == 0:
      ranks[k] = sigmas.shape[0]
      s = sigmas[-1]**2;
      while s < rel_eps / k:
        ranks[k] -= 1
        s += sigmas[-(sigmas.shape[0] - ranks[k])]**2
      ranks[k] += 1
      ranks[k] = min(ranks[k], len(sigmas))

    if k == d - 1:
      G[k] = VT[:ranks[k], :]
    else:
      G[k] = VT[:ranks[k], :].reshape((ranks[k], n_k, r_right))
    G[k-1] = np.tensordot(G[k-1], U[:, :ranks[k]] @ np.diag(sigmas[:ranks[k]]), axes=(-1, 0))
  ranks = ranks[1:]
  return G, ranks

In [167]:
shape = (20, 30, 40, 40)
i, j, k, l = np.indices(shape)
T = np.sin(i + j + k + l)

G, ranks = ttsvd(T)

A = G[0]
for k in range(1, T.ndim):
  A = np.tensordot(A, G[k], axes=(-1, 0))
print(ranks)

print(np.linalg.norm(T - A)**2 / np.linalg.norm(T)**2)

[2 2 2]
3.283603710205565e-28


In [168]:
for k in range(1, T.ndim):
  G[k] = np.ones(G[k].shape)

A = G[0]
for k in range(1, T.ndim):
  A = np.tensordot(A, G[k], axes=(-1, 0))

G = ttort(G)

B = G[0]
for k in range(1, T.ndim):
  B = np.tensordot(B, G[k], axes=(-1, 0))


print(np.linalg.norm(A - B)**2 / np.linalg.norm(T)**2)

1.4503808992059549e-30


In [181]:
shape = (20, 30, 40, 40)
i, j, k, l = np.indices(shape)
T = np.sin(i + j + k + l)

ranks = [20, 30, 40]
G, ranks = ttsvd(T, ranks=ranks)

A = G[0]
for k in range(1, T.ndim):
  A = np.tensordot(A, G[k], axes=(-1, 0))

print(ranks)
G, ranks = ttround(G)
print(ranks)

A = G[0]
for k in range(1, T.ndim):
  A = np.tensordot(A, G[k], axes=(-1, 0))

print(np.linalg.norm(T - A)**2 / np.linalg.norm(T)**2)

[20 30 40]
[2 2 2]
3.2602514933703403e-28
